![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.preprocessing import StandardScaler

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')
sample_submission = pd.read_csv('/sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/sample_submission.csv'

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data1=data.copy() # копия чтобы можно было восстановить город после get dummies
# приведем названия колонок к удобным
data.columns=['restaurant_id','city','cuisine_style','ranking',
              'price_range','number_of_reviews','reviews','url_ta','id_ta','sample','Rating']
data1.columns=['restaurant_id','city','cuisine_style','ranking',
              'price_range','number_of_reviews','reviews','url_ta','id_ta','sample','Rating']

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['number_of_reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['number_of_reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'city',], dummy_na=True)
data['city']=data1['city']

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['price_range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'
#функция для присвоения значений цен цифрам от одного до трех
import random
def price_ranges(df):
    if df == '$':
        return 1
    elif df == '$$ - $$$':
        return 2
    elif df == '$$$$':
        return 3

#функция для заполнения NaN-ов
def func(x):
    if np.isnan(x):
        return random.randint(1,3)
    else:
        return x
# сделаем новый столбец price_range в цифрах 
data['price_range_num']=data['price_range'].apply(price_ranges)
data['price_range_num']=data['price_range_num'].fillna(2.0)

In [ ]:
data['price_range_num'].value_counts() # проверим, какое распределение стало после

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

## Кухни

In [ ]:
# тут ваш код на обработку других признаков
data['cuisine_style']=data['cuisine_style'].fillna("['secret']")
data['cuisine_style']=data['cuisine_style'].apply(lambda x: eval(x))

In [ ]:
# множество с редко встречающимися кухнями (менее 0.01%) датасета
data_rare=data.copy()
data_rare=data_rare.explode('cuisine_style')
rare_cuisine_dict = (data_rare['cuisine_style'].value_counts(
)[data_rare['cuisine_style'].value_counts(normalize=True) <= 0.0001]).to_dict()  
rare_cuisine = set(rare_cuisine_dict.keys()) 
rare_cuisine

In [ ]:
# множество с остальными кухнями
data_common=data.copy()
data_common=data_common.explode('cuisine_style')
common_cuisine_dict = (data_common['cuisine_style'].value_counts(
)[data_common['cuisine_style'].value_counts(normalize=True) > 0.0001]).to_dict()
common_cuisine = set(common_cuisine_dict.keys()) 
common_cuisine

In [ ]:
cuisine_styles = []  # создаём пустой список для хранения уникальных значений кухонь
for cuisine in data['cuisine_style']: 
    for ing in cuisine:
        if not ing in cuisine_styles:
            cuisine_styles.append(ing)
cuisine_styles # список со всеми кухнями

In [ ]:
# создадим столбцы с названиями кухонь
def find_item(cell):
    if item in cell:
        return 1
    return 0

for item in common_cuisine:
    data[item] = data['cuisine_style'].apply(find_item)

data

In [ ]:
# редко стречающиеся кухни выделим в отдельную колонку
other_cuisine=[] 
for i in range(50000):
    if len(rare_cuisine.intersection(data.cuisine_style[i])) != 0:
        other_cuisine.append(1)
    else:
        other_cuisine.append(0)

len(other_cuisine) # сначала создадим список, в котором если кухня редкая - 1, если нет - 0

In [ ]:
data['other_cuisine']=other_cuisine # и добавим его колонкой в данные
data['other_cuisine'].value_counts()

In [ ]:
data['cuisine_count'] = data['cuisine_style'].apply(
    lambda x: 1 if len(x) == 0 else len(x))

## Работа с отзывами и датами в отзывах

In [ ]:
# вытащим даты в новый столбец
import re
pattern = re.compile('\d+/\d+/\d+')
data['reviews_date']=data['reviews'].apply(lambda x:pattern.findall(str(x)))

In [ ]:
# функции для отделения первой и второй даты в разные столбцы
def last_date(df):
    if len(df) > 0:
        return df[0]
    else:
        return 0


def another_date(df):
    if len(df) > 0:
        return df[-1]
    else:
        return 0

In [ ]:
#обработка дат в отзывах, пустые даты заполнены '1970-01-01'
data['last_reviews_date'] = data.reviews_date.apply(last_date)
data['first_reviews_date'] = data.reviews_date.apply(another_date)
data.last_reviews_date=pd.to_datetime(data['last_reviews_date'])
data.first_reviews_date=pd.to_datetime(data['first_reviews_date'])
data.sample(10)

In [ ]:
data['timedelta']=(data.last_reviews_date-data.first_reviews_date)
data['timedelta']=pd.to_numeric(data['timedelta'].dt.days, downcast ='integer') # добавим столбец с количеством дней между отзывами

In [ ]:
# Создадим множества со словами, которые встречаются в плохих или хороших отзывах
good_words = {'good', 'fine', 'delicious', 'best', 'wonderful', 'nice', 'better', 'top notch', 'great', 'cozy',
              'yummy', 'amazing', 'fantastic', 'wonderful', 'perfect', 'tasty', 'fab', 'worth', 'excellent', 'ace',
              'beautiful', 'lovely', 'quick', 'top', 'enjoyable', 'fabulous'}
bad_words = {'bad', 'worst', 'poor', 'awful', 'rude', 'noisy', 'horrible', 'wrong', 'blaah', 'slow', 'poor', 'rough',
             'dirty', 'overpriced', 'nothing'}

In [ ]:
data['reviews']=data['reviews'].str.lower() # приведем отзывы к нижнему регистру
data['reviews_new']=data['reviews'].apply(lambda x:re.split(r'\W+', str(x), maxsplit=0)) # и сделаем из отзывов списки слов

In [ ]:
good=[] 
for i in range(50000):
    if len(good_words.intersection(data.reviews_new[i])) != 0:
        good.append(1)
    else:
        good.append(0)
len(good) # список с 0 и 1 (если в отзыве слова из "хорошего" списка)

In [ ]:
bad=[] 
for i in range(50000):
    if len(bad_words.intersection(data.reviews_new[i])) != 0:
        bad.append(1)
    else:
        bad.append(0)
len(bad) # список с 0 и 1 (если в отзыве слова из "плохого" списка)

In [ ]:
data['good_reviews']=good # и добавим его колонкой в данные
data['good_reviews'].value_counts()

In [ ]:
data['bad_reviews']=bad # и добавим его колонкой в данные
data['bad_reviews'].value_counts()

### Добавим данные

In [ ]:
# словарь с плотностью населения
density_of_city = {'London': 5137, 'Paris': 20781, 'Madrid': 5390, 'Barcelona': 15779, 'Berlin': 4463, 'Milan': 7385,
                    'Rome': 2229, 'Prague': 2473, 'Lisbon': 5066, 'Vienna': 4438, 'Amsterdam': 4457, 'Brussels': 4439,
                    'Hamburg': 2388, 'Munich': 4686, 'Lyon': 10023, 'Stockholm': 5114, 'Budapest': 3306, 'Warsaw': 3461,
                    'Dublin': 4588, 'Copenhagen': 4514, 'Athens': 17026, 'Edinburgh': 4136, 'Zurich': 4574, 'Oporto': 6946,
                    'Geneva': 6816, 'Krakow': 2325, 'Oslo': 1527, 'Helsinki': 2739, 'Bratislava': 1140, 'Luxembourg': 245,
                    'Ljubljana': 1759}

In [ ]:
# сначала делаем DF из словаря, и добавляем плотность городов
density_of_city_df=pd.DataFrame(list(density_of_city.items()),
                           columns=['city','density_of_city'])
data=data.merge(density_of_city_df, how='left',left_on='city', right_on='city')

In [ ]:
# добавим колонку с количеством ресторанов (по id) возожно, как-то влияет большая это сеть один ресторан
count_rests_dict = data.restaurant_id.value_counts().to_dict()
count_rests = pd.DataFrame(list(count_rests_dict.items()),
                           columns=['restaurant_id','count_rests'])
count_rests

In [ ]:
# добавим колонку с общим количеством ресторанов в городе
count_in_city_dict = data.city.value_counts().to_dict()
count_in_city = pd.DataFrame(list(count_in_city_dict.items()),
                           columns=['city','count_in_city'])
count_in_city

In [ ]:
data=data.merge(count_rests, how='left',left_on='restaurant_id', right_on='restaurant_id')
data=data.merge(count_in_city, how='left',left_on='city', right_on='city')
data

In [ ]:
# нормируем признак  ranking
a=np.array(data.ranking)
a=np.reshape(a,(50000,1)) # матрица из значений для нормировки

scaler = StandardScaler()
ranking_norm=scaler.fit_transform(a) # нормированные значения

ranking_norm

In [ ]:
data=data.join(pd.DataFrame(ranking_norm), rsuffix='_') # добавляем в наши данные
data

In [ ]:
data = data.rename(
    columns={0: 'ranking_norm'}) #  переименуем корректно столбец
data

In [ ]:
# добавим колонку с средним значением ranking_norm в городе
ranking_city=data.pivot_table(values=['ranking_norm'],index=['city'],aggfunc='mean',fill_value = 0)
ranking_city

In [ ]:
# добавим колонку с максимальным значением ranking_norm в городе
ranking_city_max=data.pivot_table(values=['ranking_norm'],index=['city'],aggfunc='max',fill_value = 0)
ranking_city_max

In [ ]:
# добавим колонку с минимальным значением ranking_norm в городе
ranking_city_min=data.pivot_table(values=['ranking_norm'],index=['city'],aggfunc='min',fill_value = 0)
ranking_city_min

In [ ]:
data=data.merge(ranking_city, how='left',left_on='city', right_on='city')

data = data.rename(
    columns={'ranking_norm_y': 'ranking_mean', 'ranking_norm_x': 'ranking_norm'})
data

In [ ]:
data=data.merge(ranking_city_max, how='left',left_on='city', right_on='city')

data = data.rename(
    columns={'ranking_norm_y': 'ranking_city_max', 'ranking_norm_x': 'ranking_norm'})
data

In [ ]:
data

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

In [ ]:
data.corr().Rating.sort_values(ascending=False).to_frame()

# Data Preprocessing


In [ ]:
data.drop(['restaurant_id','city','cuisine_style','reviews','url_ta','id_ta','price_range','reviews_date','last_reviews_date',
           'first_reviews_date', 'reviews_new'], axis = 1, inplace=True)

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    df_output.columns=['restaurant_id','city','cuisine_style','ranking',
                       'price_range','number_of_reviews','reviews','url_ta','id_ta','sample','Rating']
    # ################### 1. Предобработка ############################################################## 
 
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['number_of_reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'city',], dummy_na=True)
    df_output['city']=data1['city']
    
    df_output['price_range_num']=df_output['price_range'].apply(price_ranges)
    df_output['price_range_num']=df_output['price_range_num'].fillna(2.0)
    
    df_output['cuisine_style']=df_output['cuisine_style'].fillna("['secret']")
    df_output['cuisine_style']=df_output['cuisine_style'].apply(lambda x: eval(x))
    
    df_output['other_cuisine']=other_cuisine
    
    df_output['cuisine_count'] = df_output['cuisine_style'].apply(lambda x: 1 if len(x) == 0 else len(x))
    
    df_output['reviews_date']=df_output['reviews'].apply(lambda x:pattern.findall(str(x)))
    df_output['last_reviews_date'] = df_output.reviews_date.apply(last_date)
    df_output['first_reviews_date'] = df_output.reviews_date.apply(another_date)
    df_output.last_reviews_date=pd.to_datetime(df_output['last_reviews_date'])
    df_output.first_reviews_date=pd.to_datetime(df_output['first_reviews_date'])
    df_output['timedelta']=(df_output.last_reviews_date-df_output.first_reviews_date)
    df_output['timedelta']=pd.to_numeric(df_output['timedelta'].dt.days, downcast ='integer')
    
    df_output['good_reviews']=good 
    df_output['bad_reviews']=bad 
    
    df_output=df_output.merge(density_of_city_df, how='left',left_on='city', right_on='city')
    
    df_output=df_output.merge(count_rests, how='left',left_on='restaurant_id', right_on='restaurant_id')
    df_output=df_output.merge(count_in_city, how='left',left_on='city', right_on='city')
    
    df_output=df_output.join(pd.DataFrame(ranking_norm), rsuffix='_')
    df_output = df_output.rename(columns={0: 'ranking_norm'}) #  переименуем корректно столбец
    
    df_output=df_output.merge(ranking_city, how='left',left_on='city', right_on='city')
    df_output = df_output.rename(columns={'ranking_norm_y': 'ranking_mean', 'ranking_norm_x': 'ranking_norm'})
    
    df_output=df_output.merge(ranking_city_max, how='left',left_on='city', right_on='city')
    df_output = df_output.rename(columns={'ranking_norm_y': 'ranking_city_max', 'ranking_norm_x': 'ranking_norm'})
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    df_output.drop(['restaurant_id','city','cuisine_style','reviews','url_ta','id_ta','price_range','reviews_date','last_reviews_date',
               'first_reviews_date'], axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = data 
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
